In [1]:
!rm -rf data
!wget http://kdd.ics.uci.edu/databases/kddcup99/task.html -O task.html
!mkdir data
!wget http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data.gz -O data/kddcup.data.gz 
!wget http://kdd.ics.uci.edu/databases/kddcup99/kddcup.names -O data/kddcup.names
!wget http://kdd.ics.uci.edu/databases/kddcup99/kddcup.names  -O data/kddcup.names
!wget http://kdd.ics.uci.edu/databases/kddcup99/training_attack_types -O data/training_attack_types
!gunzip data/kddcup.data.gz

--2021-12-24 10:00:37--  http://kdd.ics.uci.edu/databases/kddcup99/task.html
Resolving kdd.ics.uci.edu (kdd.ics.uci.edu)... 128.195.1.86
Connecting to kdd.ics.uci.edu (kdd.ics.uci.edu)|128.195.1.86|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10277 (10K) [text/html]
Saving to: ‘task.html’

task.html           100%[===================>]  10.04K  --.-KB/s    in 0.001s  

2021-12-24 10:00:37 (8.95 MB/s) - ‘task.html’ saved [10277/10277]

--2021-12-24 10:00:37--  http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data.gz
Resolving kdd.ics.uci.edu (kdd.ics.uci.edu)... 128.195.1.86
Connecting to kdd.ics.uci.edu (kdd.ics.uci.edu)|128.195.1.86|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18115902 (17M) [application/x-gzip]
Saving to: ‘data/kddcup.data.gz’

data/kddcup.data.gz 100%[===================>]  17.28M  4.34MB/s    in 4.7s    

2021-12-24 10:00:42 (3.67 MB/s) - ‘data/kddcup.data.gz’ saved [18115902/18115902]

--2021-12-24 10:00:42--

,back,buffer_overflow,ftp_write,guess_passwd,imap,ipsweep,land,loadmodule,multihop,neptune,...,phf,pod,portsweep,rootkit,satan,smurf,spy,teardrop,warezclient,warezmaster
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4898426,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4898427,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4898428,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4898429,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [1]:
import os
import pandas as pd
import numpy as np
# import plotly.graph_objects as go 
import sklearn 
from sklearn.linear_model import LogisticRegression
import imblearn
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from pandas_profiling import ProfileReport
import dask 

In [2]:
from dask_ml.model_selection import train_test_split
from dask_ml.linear_model import LogisticRegression


In [9]:
DATA_DIR = 'data'

COLS = [
'duration', 
'protocol_type', 
'service', 
'flag', 
'src_bytes', 
'dst_bytes', 
'land', 
'wrong_fragment', 
'urgent', 
'hot', 
'num_failed_logins', 
'logged_in', 
'num_compromised', 
'root_shell', 
'su_attempted', 
'num_root', 
'num_file_creations', 
'num_shells', 
'num_access_files', 
'num_outbound_cmds', 
'is_host_login', 
'is_guest_login', 
'count', 
'srv_count', 
'serror_rate', 
'srv_serror_rate', 
'rerror_rate', 
'srv_rerror_rate', 
'same_srv_rate', 
'diff_srv_rate', 
'srv_diff_host_rate', 
'dst_host_count', 
'dst_host_srv_count', 
'dst_host_same_srv_rate', 
'dst_host_diff_srv_rate', 
'dst_host_same_src_port_rate', 
'dst_host_srv_diff_host_rate', 
'dst_host_serror_rate', 
'dst_host_srv_serror_rate', 
'dst_host_rerror_rate', 
'dst_host_srv_rerror_rate',
'normal'
]

df = dask.dataframe.read_csv(os.path.join(DATA_DIR, 'kddcup.data'), sep=',', header=None, names=COLS)
df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,normal
0,0,tcp,http,SF,215,45076,0,0,0,0,...,0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,162,4528,0,0,0,0,...,1,1.0,0.0,1.00,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,236,1228,0,0,0,0,...,2,1.0,0.0,0.50,0.0,0.0,0.0,0.0,0.0,normal.
3,0,tcp,http,SF,233,2032,0,0,0,0,...,3,1.0,0.0,0.33,0.0,0.0,0.0,0.0,0.0,normal.
4,0,tcp,http,SF,239,486,0,0,0,0,...,4,1.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,normal.


In [10]:
df.normal.unique()

Dask Series Structure:
npartitions=1
    object
       ...
Name: normal, dtype: object
Dask Name: unique-agg, 39 tasks

In [28]:
df['protocol_type'] = df['protocol_type'].astype('category')
df['service'] = df['service'].astype('category')
df['flag'] = df['flag'].astype('category')

In [29]:
protocol_type = dask.dataframe.reshape.get_dummies(df.protocol_type.cat.as_known())
service = dask.dataframe.reshape.get_dummies(df.service.cat.as_known())
flag = dask.dataframe.reshape.get_dummies(df.flag.cat.as_known())

In [30]:
import dask 
import dask.dataframe as dd
from dask_ml.preprocessing import LabelEncoder

le = LabelEncoder()

df['normal_enc'] = le.fit_transform(df.normal)

In [33]:
df = df.drop(['protocol_type', 'service', 'flag'], axis=1)

In [46]:
data = dask.dataframe.multi.concat([df, protocol_type, service, flag], axis=1)
data=data.drop('normal',axis=1)

/home/aadi/miniconda3/lib/python3.9/site-packages/dask/dataframe/multi.py:1228: UserWarning: Concatenating dataframes with unknown divisions.
We're assuming that the indices of each dataframes are 
 aligned. This assumption is not generally safe.
  warnings.warn(


In [10]:
clf = LogisticRegression()
X_train, X_test, y_train, y_test = train_test_split(df.drop('normal', axis=1), df['normal'])

clf.fit(X_train, y_train)

ValueError: could not convert string to float: 'icmp'